In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.exceptions import ConvergenceWarning
import warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

train_y = train.Transported
# features = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
# train_x = train[features]
# test_x = test[features]

In [12]:
# # Encode categorical features using one-hot encoding
# train_x_encoded = pd.get_dummies(train_x)
# test_x_encoded = pd.get_dummies(test_x)

# # Ensure train/test have same columns (important!)
# train_x_encoded, test_x_encoded = train_x_encoded.align(test_x_encoded, join='left', axis=1, fill_value=0)

# # Fill all NaNs with 0 (or use a more suitable value)
# train_x_encoded = train_x_encoded.fillna(0)
# test_x_encoded = test_x_encoded.fillna(0)

In [13]:
# Feature engineering
def feature_engineering(df):
    df = df.copy()
    df['CabinDeck'] = df['Cabin'].apply(lambda x: str(x).split('/')[0] if pd.notnull(x) else x)
    df['CabinSide'] = df['Cabin'].apply(lambda x: str(x).split('/')[-1] if pd.notnull(x) else x)
    df['TotalSpending'] = df[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)
    return df

train = feature_engineering(train)
test = feature_engineering(test)

# Define features
features = ['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP',
            'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
            'CabinDeck', 'CabinSide', 'TotalSpending']
train_x = train[features]
test_x = test[features]

In [14]:
# Define categorical and numeric features
categorical_cols = [col for col in train_x.columns if train_x[col].dtype == "object"]
numeric_cols = [col for col in train_x.columns if train_x[col].dtype in ['int64', 'float64']]

# Preprocessing pipelines
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
])

# Full preprocessor
preprocessor = ColumnTransformer([
    ('cat', categorical_pipeline, categorical_cols),
    ('num', numeric_pipeline, numeric_cols)
])

In [15]:
# best_score = -np.inf
# best_leaf_nodes = None
# scores = {}
# best_params = {}

# # Store best scores
# best_scores = {}

# # RandomForestClassifier tuning
# best_score = -np.inf
# for n in [40]:
#     for d in [13]:
#         for l in [69]:
#             model = Pipeline([
#                 ("preprocessor", preprocessor),
#                 ("model", RandomForestClassifier(n_estimators=n, max_depth=d, max_leaf_nodes=l, random_state=1))
#             ])
#             score = cross_val_score(model, train_x, train_y, cv=5, scoring="accuracy").mean()
#             if score > best_score:
#                 best_score = score
#                 best_scores["RandomForestClassifier"] = (score, {"n_estimators": n, "max_depth": d, "max_leaf_nodes": l})

# # RandomForestRegressor tuning
# best_score = -np.inf
# for n in [60]:
#     for d in [8]:
#         for l in [97]:
#             model = Pipeline([
#                 ("preprocessor", preprocessor),
#                 ("model", RandomForestRegressor(n_estimators=n, max_depth=d, max_leaf_nodes=l, random_state=1))
#             ])
#             score = cross_val_score(model, train_x, train_y.astype(int), cv=5, scoring="r2").mean()
#             if score > best_score:
#                 best_score = score
#                 best_scores["RandomForestRegressor"] = (score, {"n_estimators": n, "max_depth": d, "max_leaf_nodes": l})

# # LogisticRegression tuning
# best_score = -np.inf
# logistic_params = [
#     (1160, 'lbfgs', 0.003)
# ]

# for max_iter, solver, C in logistic_params:
#     model = Pipeline([
#         ("preprocessor", preprocessor),
#         ("model", LogisticRegression(max_iter=max_iter, solver=solver, C=C, random_state = 1))
#     ])
#     score = cross_val_score(model, train_x, train_y, cv=5, scoring="accuracy").mean()
#     if score > best_score:
#         best_score = score
#         best_scores["LogisticRegression"] = (score, {"max_iter": max_iter, "solver": solver, "C": C})

# # KNeighbors tuning
# best_score = -np.inf
# for k in [15]:
#     model = Pipeline([
#         ("preprocessor", preprocessor),
#         ("model", KNeighborsClassifier(n_neighbors=k))
#     ])
#     score = cross_val_score(model, train_x, train_y, cv=5, scoring="accuracy").mean()
#     if score > best_score:
#         best_score = score
#         best_scores["KNeighborsClassifier"] = (score, {"n_neighbors": k})

# print(best_scores)

In [16]:
rfcmodel = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier(
        n_estimators=40, max_depth=13, max_leaf_nodes=69, random_state=1
    ))
])
# rfcmodel.fit(train_x, train_y)

# test_preds = rfcmodel.predict(test_x)


logmodel = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', LogisticRegression(max_iter=1160, solver='lbfgs', C = 0.003))
])
# logmodel.fit(train_x, train_y)

# test_preds = logmodel.predict(test_x)


knnmodel = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', KNeighborsClassifier(n_neighbors=15))
])
# knnmodel.fit(train_x, train_y)

# test_preds = knnmodel.predict(test_x)


# rfrmodel = Pipeline([
#     ('preprocessing', preprocessor),
#     ('regressor', RandomForestRegressor(n_estimators=60, max_depth=8, max_leaf_nodes=97, random_state=1))
# ])
# rfrmodel.fit(train_x, train_y)

# test_preds = rfrmodel.predict(test_x) > 0.5

In [17]:
# Hard voting
hard_voting = VotingClassifier(estimators=[
    ('rfc', rfcmodel),
    ('log', logmodel),
    ('knn', knnmodel)
], voting='hard')

hard_voting.fit(train_x, train_y)
hard_preds = hard_voting.predict(test_x)

# # Soft voting
# soft_voting = VotingClassifier(estimators=[
#     ('rfc', rfcmodel),
#     ('log', logmodel),
#     ('knn', knnmodel)
# ], voting='soft')

# soft_voting.fit(train_x, train_y)
# soft_preds = soft_voting.predict(test_x)

# Stacking
stacking = StackingClassifier(
    estimators=[
        ('knn', knnmodel),
        ('rfc', rfcmodel)
    ],
    final_estimator=LogisticRegression(max_iter=1160, solver='lbfgs', C = 0.003),  
    n_jobs=-1
)

stacking.fit(train_x, train_y)
stacked_preds = stacking.predict(test_x)

In [18]:
output = pd.DataFrame({'PassengerId': test.PassengerId,
                       'Transported': hard_preds})
output.to_csv('submission.csv', index=False)